In [ ]:
%pip install transformers
%pip install torch
%pip install PyTorch
%pip install import_ipynb

In [3]:
import import_ipynb
import cleanData as datab

Empty DataFrame
Columns: [user_id, text]
Index: []


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [5]:
model_name = "tabularisai/robust-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


In [6]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(probabilities, dim=-1).item()
        sentiment_map = {
            0: "Very Negative",
            1: "Negative",
            2: "Neutral",
            3: "Positive",
            4: "Very Positive"
        }
        return sentiment_map[predicted_class]


In [10]:
datab.dataset_df["sentiment"] = datab.dataset_df["text"].apply(predict_sentiment)
print(datab.dataset_df[["text", "sentiment"]].head())
 

                                                text      sentiment
0  Great staff always helps and always nice. Alwa...  Very Positive
1  After my ROTD  yesterday of a different Sweet ...        Neutral
2  Our family returned for breakfast again this w...       Positive
3  If I could give it a zero, I would. I order a ...  Very Negative
4  Id you haven't been to the Smoothie King cente...       Positive


In [18]:
datab.dataset_df.to_json("hugging_face_sentiment_analysis.json", orient="records", lines=True)
